# Compile Observation Statistics

I want to compile a bunch of information about each observation for the eventual table in my paper. Things to compile:

- Star
- RA/DEC
- UT date
- Julian Date
- Observation Time
- S/N ratio

In [ ]:
import os
from astropy.io import fits
import HelperFunctions
import glob
import h5py
import logging
import numpy as np
import pandas as pd
from astropy.time import Time

logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [ ]:
home = os.environ['HOME']
basedir = os.path.join(home, 'School', 'Research')

inst_dirs = dict(TS23=os.path.join(basedir, 'McDonaldData'),
                 HRS=os.path.join(basedir, 'HET_data'),
                 CHIRON=os.path.join(basedir, 'CHIRON_data'),
                 IGRINS=os.path.join(basedir, 'IGRINS_data'))

In [ ]:
inst_file_lists = {}
for inst in inst_dirs.keys():
    file_list = []
    logging.info('Instrument: {}'.format(inst))
    # Get the CCF.hdf5 file, and loop through to get the original filename for every CCF
    with h5py.File(os.path.join(inst_dirs[inst], 'Cross_correlations', 'CCF.hdf5'), 'r') as infile:
        for star, star_grp in infile.iteritems():
            logging.debug('Reading in datasets for star {}'.format(star))
            for date, date_grp in star_grp.iteritems():
                for ds_name, ds in date_grp.iteritems():
                    fname = ds.attrs['fname']
                    if fname not in file_list:
                        file_list.append(fname)
                    else:
                        break
    inst_file_lists[inst] = file_list

In [ ]:
def get_snr(fname, inst):
    orders = HelperFunctions.ReadExtensionFits(fname)
    wlen = 2200 if inst == 'IGRINS' else 675
    ordernum = HelperFunctions.FindOrderNums(orders, [wlen])[0]
    order = orders[ordernum]
    if inst == 'IGRINS':
        return np.median(np.std(order.y))
    return np.median(order.y/order.err)

In [ ]:
starname = []
ut = []
jd = []
exptime = []
snr = []
ra = []
dec = []
instrument = []
bad_fnames = []
for inst in inst_file_lists.keys():
    for fname in inst_file_lists[inst]:
        #print(inst, fname)
        full_fname = os.path.join(inst_dirs[inst], fname)
        header = fits.getheader(full_fname)
        starname.append(header['OBJECT'])
        ut.append(header['UT'])
        jd.append(header['JD'])
        try:
            exptime.append(header['EXPTIME'])
        except KeyError:
            print('No Exptime keyword in file {}'.format(fname))
            bad_fnames.append(full_fname)
            continue
        snr.append(get_snr(full_fname, inst))
        ra.append(header['RA'])
        dec.append(header['DEC'])
        instrument.append(inst)

stats = pd.DataFrame(data=dict(star=starname, instrument=instrument, ut=ut, jd=jd,
                               exptime=exptime, snr=snr, ra=ra, dec=dec))

In [ ]:
from astropy.time import Time
def get_ut_date(jd):
    t = Time(jd, format='jd', scale='utc')
    dt = t.datetime
    return '{}-{:02d}-{:02d}'.format(dt.year, dt.month, dt.day)
stats['date'] = stats.jd.map(get_ut_date)
stats.head()

In [ ]:
col_order = ['star', 'ra', 'dec', 'date', 'ut', 'jd', 'instrument', 'exptime', 'snr']
stats[col_order].to_csv('data/SampleObservations.csv', index=False)